In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import os

# ------------------- تنظیمات مسیر -------------------
base_path = r"C:\BI"   # مسیر اصلی

input_file = os.path.join(base_path, "lube_oil_system_data_g11.xlsx")
output_file = os.path.join(base_path, "lube_oil_system_anomalies_g11.xlsx")

# بررسی وجود فایل ورودی
if not os.path.exists(input_file):
    raise FileNotFoundError(f"فایل پیدا نشد!\nمسیر مورد نظر: {input_file}")

# ------------------- پارامترهای DBSCAN -------------------
eps = 0.5           # می‌تونی بعداً بین 0.3 تا 1.0 تست کنی
min_samples = 10    # حداقل نقاط برای تشکیل هسته خوشه

# ------------------- خواندن داده -------------------
print("در حال خواندن فایل از مسیر:")
print(input_file)
df = pd.read_excel(input_file)

# پیدا کردن ستون‌های سنسور (آن‌هایی که با AssetID_ شروع می‌شن و با عدد تمام می‌شن)
sensor_columns = [col for col in df.columns if col.startswith('AssetID_') and col.split('_')[-1].isdigit()]

print(f"\nستون‌های سنسور شناسایی شده ({len(sensor_columns)} تا):")
print(sensor_columns)

if len(sensor_columns) == 0:
    raise ValueError("هیچ ستون سنسوری با نام AssetID_XXXX پیدا نشد!")

# ستون‌های شناسه و زمانی که می‌خواهیم در خروجی نگه داریمid, RecordDate, RecordTime
id_columns = ['id', 'RecordDate', 'RecordTime']
available_id_cols = [col for col in id_columns if col in df.columns]

# ------------------- پیش‌پردازش داده‌های سنسور -------------------
X = df[sensor_columns].copy()

# پر کردن مقادیر گمشده (DBSCAN با NaN کار نمی‌کنه)
X = X.fillna(X.mean())          # می‌تونی به X.median() یا استراتژی دیگه تغییر بدی

# استانداردسازی (خیلی مهم برای DBSCAN)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nتعداد رکوردهای آماده برای مدل: {X_scaled.shape[0]}")

# ------------------- اجرای DBSCAN -------------------
print("در حال اجرای الگوریتم DBSCAN...")
db = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
labels = db.fit_predict(X_scaled)

# آمار سریع
n_anomalies = np.sum(labels == -1)
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

print(f"تعداد خوشه‌های تشکیل شده: {n_clusters}")
print(f"تعداد انومالی‌های تشخیص داده شده: {n_anomalies} رکورد ({n_anomalies/len(labels)*100:.2f}%)")

# ------------------- استخراج انومالی‌ها -------------------
anomaly_indices = df.index[labels == -1]  # ایندکس‌های اصلی در دیتافریم اصلی
anomalies_df = df.loc[anomaly_indices, available_id_cols + sensor_columns].copy()

# اضافه کردن برچسب DBSCAN (همه -1 هستند)
anomalies_df['DBSCAN_Label'] = -1

# مرتب‌سازی بر اساس تاریخ و زمان (اگر موجود باشد)
if 'RecordDate' in anomalies_df.columns and 'RecordTime' in anomalies_df.columns:
    anomalies_df['datetime_temp'] = pd.to_datetime(
        anomalies_df['RecordDate'].astype(str) + ' ' + anomalies_df['RecordTime'].astype(str),
        errors='coerce'
    )
    anomalies_df = anomalies_df.sort_values('datetime_temp').drop(columns='datetime_temp')
else:
    anomalies_df = anomalies_df.sort_index()

# ------------------- ذخیره فایل انومالی در مسیر C:\BI -------------------
print("\nدر حال ذخیره فایل انومالی‌ها...")
anomalies_df.to_excel(output_file, index=False)

print(f"انومالی‌ها با موفقیت ذخیره شدند!")
print(f"تعداد انومالی‌ها: {len(anomalies_df)} ردیف")
print(f"مسیر فایل خروجی:\n{output_file}")

در حال خواندن فایل از مسیر:
C:\BI\lube_oil_system_data_g11.xlsx

ستون‌های سنسور شناسایی شده (7 تا):
['AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344', 'AssetID_8346', 'AssetID_9286', 'AssetID_9287']

تعداد رکوردهای آماده برای مدل: 10319
در حال اجرای الگوریتم DBSCAN...
تعداد خوشه‌های تشکیل شده: 13
تعداد انومالی‌های تشخیص داده شده: 482 رکورد (4.67%)

در حال ذخیره فایل انومالی‌ها...
انومالی‌ها با موفقیت ذخیره شدند!
تعداد انومالی‌ها: 482 ردیف
مسیر فایل خروجی:
C:\BI\lube_oil_system_anomalies_g11.xlsx


In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import os
from collections import Counter

# ------------------- تنظیمات مسیر -------------------
base_path = r"C:\BI"
input_file = os.path.join(base_path, "lube_oil_system_data_g11.xlsx")
output_file = os.path.join(base_path, "lube_oil_system_anomalies_g11.xlsx")

# بررسی وجود فایل ورودی
if not os.path.exists(input_file):
    raise FileNotFoundError(f"فایل ورودی پیدا نشد!\nمسیر: {input_file}")

# ------------------- پارامترهای DBSCAN -------------------
eps = 0.5
min_samples = 10

# ------------------- خواندن داده -------------------
print("در حال خواندن فایل اکسل...")
df = pd.read_excel(input_file)

# ستون‌های سنسور
sensor_columns = [col for col in df.columns if col.startswith('AssetID_') and col.split('_')[-1].isdigit()]
print(f"\nستون‌های سنسور شناسایی شده ({len(sensor_columns)} تا): {sensor_columns}")

if len(sensor_columns) == 0:
    raise ValueError("ستون سنسوری پیدا نشد!")

id_columns = ['id', 'RecordDate', 'RecordTime']
available_id_cols = [col for col in id_columns if col in df.columns]

# ------------------- پیش‌پردازش -------------------
X = df[sensor_columns].copy()
X = X.fillna(X.mean())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nتعداد کل رکوردهای آماده برای خوشه‌بندی: {len(X_scaled):,} رکورد")

# ------------------- اجرای DBSCAN -------------------
print(f"\nاجرای DBSCAN با eps={eps} و min_samples={min_samples}...")
db = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
labels = db.fit_predict(X_scaled)

# ------------------- آمار خوشه‌ها -------------------
label_counts = Counter(labels)
n_clusters = len(label_counts) - (1 if -1 in label_counts else 0)
n_noise = label_counts[-1] if -1 in label_counts else 0

print("\n" + "="*60)
print("               خلاصه نتایج خوشه‌بندی DBSCAN")
print("="*60)
print(f"{'تعداد کل رکورد ها':<25}: {len(labels):,}")
print(f"{'تعداد انومالی (نویز)':<25}: {n_noise:,}  ({n_noise/len(labels)*100:6.2f}%)")
print(f"{'تعداد خوشه‌های اصلی':<25}: {n_clusters:,}")
print("-" * 60)

# نمایش تعداد اعضا در هر خوشه (به ترتیب اندازه)
cluster_sizes = []
for label, count in label_counts.items():
    if label != -1:  # فقط خوشه‌های واقعی
        cluster_sizes.append((label, count))

# مرتب‌سازی بر اساس تعداد اعضا (نزولی)
cluster_sizes.sort(key=lambda x: x[1], reverse=True)

print(f"{'خوشه':<10} {'تعداد اعضا':<15} {'درصد از کل'}")
print("-" * 60)
for i, (label, count) in enumerate(cluster_sizes, 1):
    percentage = count / len(labels) * 100
    print(f"خوشه {label:<6} {count:>10,} رکورد    ({percentage:6.2f}%)")
    if i >= 20:  # فقط 20 خوشه بزرگ را نشان بده تا شلوغ نشود
        if len(cluster_sizes) > 20:
            print(f"    ... و {len(cluster_sizes) - 20} خوشه کوچک دیگر")
        break

print("="*60)

# ------------------- استخراج انومالی‌ها -------------------
anomaly_mask = labels == -1
anomaly_indices = df.index[anomaly_mask]

anomalies_df = df.loc[anomaly_indices, available_id_cols + sensor_columns].copy()
anomalies_df['DBSCAN_Label'] = -1
anomalies_df['Cluster_Size'] = n_noise  # فقط برای اطلاعات

# مرتب‌سازی بر اساس تاریخ و زمان
if 'RecordDate' in anomalies_df.columns and 'RecordTime' in anomalies_df.columns:
    anomalies_df['datetime_temp'] = pd.to_datetime(
        anomalies_df['RecordDate'].astype(str) + ' ' + anomalies_df['RecordTime'].astype(str),
        errors='coerce'
    )
    anomalies_df = anomalies_df.sort_values('datetime_temp').drop(columns='datetime_temp')


در حال خواندن فایل اکسل...

ستون‌های سنسور شناسایی شده (7 تا): ['AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344', 'AssetID_8346', 'AssetID_9286', 'AssetID_9287']

تعداد کل رکوردهای آماده برای خوشه‌بندی: 10,319 رکورد

اجرای DBSCAN با eps=0.5 و min_samples=10...

               خلاصه نتایج خوشه‌بندی DBSCAN
تعداد کل رکورد ها        : 10,319
تعداد انومالی (نویز)     : 482  (  4.67%)
تعداد خوشه‌های اصلی      : 13
------------------------------------------------------------
خوشه       تعداد اعضا      درصد از کل
------------------------------------------------------------
خوشه 0           4,308 رکورد    ( 41.75%)
خوشه 6           2,789 رکورد    ( 27.03%)
خوشه 2           1,380 رکورد    ( 13.37%)
خوشه 12          1,211 رکورد    ( 11.74%)
خوشه 9              29 رکورد    (  0.28%)
خوشه 1              22 رکورد    (  0.21%)
خوشه 4              19 رکورد    (  0.18%)
خوشه 7              16 رکورد    (  0.16%)
خوشه 11             16 رکورد    (  0.16%)
خوشه 8              14 رکورد    (  0

In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import os
from collections import Counter

# ------------------- تنظیمات مسیر -------------------
base_path = r"C:\BI"
input_file = os.path.join(base_path, "lube_oil_system_data_g11.xlsx")
output_file = os.path.join(base_path, "lube_oil_system_anomalies_g11.xlsx")

if not os.path.exists(input_file):
    raise FileNotFoundError(f"فایل ورودی پیدا نشد!\nمسیر: {input_file}")

# ------------------- پارامترهای DBSCAN -------------------
eps = 0.5
min_samples = 10
MIN_CLUSTER_SIZE = 15  # حداقل اندازه خوشه برای "عادی" در نظر گرفتن

# ------------------- خواندن داده -------------------
print("در حال خواندن فایل اکسل...")
df = pd.read_excel(input_file)

# ستون‌های سنسور
sensor_columns = [col for col in df.columns if col.startswith('AssetID_') and col.split('_')[-1].isdigit()]
print(f"\nستون‌های سنسور شناسایی شده ({len(sensor_columns)} تا): {sensor_columns}")

if len(sensor_columns) == 0:
    raise ValueError("هیچ ستون سنسوری پیدا نشد!")

id_columns = ['id', 'RecordDate', 'RecordTime']
available_id_cols = [col for col in id_columns if col in df.columns]

# ------------------- پیش‌پردازش -------------------
X = df[sensor_columns].copy()
X = X.fillna(X.mean())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nتعداد کل رکوردهای آماده: {len(X_scaled):,} رکورد")

# ------------------- اجرای DBSCAN -------------------
print(f"\nاجرای DBSCAN با eps={eps} و min_samples={min_samples}...")
db = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
labels = db.fit_predict(X_scaled)

# ------------------- تحلیل خوشه‌ها -------------------
label_counts = Counter(labels)
n_total = len(labels)

# جدا کردن نویز و خوشه‌ها
noise_count = label_counts.get(-1, 0)
cluster_labels = [lbl for lbl in label_counts.keys() if lbl != -1]
cluster_sizes = [(lbl, label_counts[lbl]) for lbl in cluster_labels]

# خوشه‌های بزرگ (عادی) و خوشه‌های کوچک (انومالی)
large_clusters = [ (lbl, cnt) for lbl, cnt in cluster_sizes if cnt >= MIN_CLUSTER_SIZE ]
small_clusters = [ (lbl, cnt) for lbl, cnt in cluster_sizes if cnt < MIN_CLUSTER_SIZE ]

n_large_clusters = len(large_clusters)
n_small_clusters = len(small_clusters)
small_cluster_points = sum(cnt for _, cnt in small_clusters)

# مجموع انومالی‌ها = نویز + خوشه‌های خیلی کوچک
total_anomalies = noise_count + small_cluster_points

# ------------------- نمایش خلاصه -------------------
print("\n" + "="*70)
print("                  خلاصه نتایج نهایی (با قوانین جدید)")
print("="*70)
print(f"{'کل داده‌ها':<30}: {n_total:,}")
print(f"{'نویز DBSCAN (برچسب -1)':<30}: {noise_count:,} رکورد")
print(f"{'خوشه‌های کوچک (< {MIN_CLUSTER_SIZE} عضو)':<30}: {n_small_clusters} خوشه → {small_cluster_points:,} رکورد")
print(f"{'مجموع انومالی‌ها':<30}: {total_anomalies:,} رکورد ({total_anomalies/n_total*100:5.2f}%)")
print(f"{'خوشه‌های اصلی (عادی ≥ {MIN_CLUSTER_SIZE} عضو)':<30}: {n_large_clusters} خوشه")
print("-"*70)

# نمایش خوشه‌های اصلی (عادی)
if n_large_clusters > 0:
    print(f"{'خوشه اصلی':<12} {'تعداد اعضا':<15} {'درصد از کل'}")
    print("-"*70)
    large_clusters_sorted = sorted(large_clusters, key=lambda x: x[1], reverse=True)
    for i, (lbl, cnt) in enumerate(large_clusters_sorted, 1):
        perc = cnt / n_total * 100
        print(f"خوشه {lbl:<8} {cnt:>10,} رکورد    ({perc:6.2f}%)")
        if i >= 15 and len(large_clusters_sorted) > 15:
            print(f"    ... و {len(large_clusters_sorted) - 15} خوشه دیگر")
            break
else:
    print("هیچ خوشه اصلی (عادی) پیدا نشد!")

print("="*70)

# ------------------- استخراج انومالی‌ها (نویز + خوشه‌های کوچک) -------------------
# ایندکس‌هایی که انومالی هستند
anomaly_mask = np.isin(labels, [-1] + [lbl for lbl, cnt in small_clusters])  # نویز + خوشه‌های کوچک
anomaly_indices = df.index[anomaly_mask]

anomalies_df = df.loc[anomaly_indices, available_id_cols + sensor_columns].copy()

# اضافه کردن دلیل انومالی بودن
anomalies_df['DBSCAN_Label'] = labels[anomaly_mask]
anomalies_df['Anomaly_Reason'] = anomalies_df['DBSCAN_Label'].apply(
    lambda x: 'Noise (-1)' if x == -1 else f'Small Cluster (size={label_counts[x]})'
)

# مرتب‌سازی بر اساس زمان
if 'RecordDate' in anomalies_df.columns and 'RecordTime' in anomalies_df.columns:
    anomalies_df['datetime_temp'] = pd.to_datetime(
        anomalies_df['RecordDate'].astype(str) + ' ' + anomalies_df['RecordTime'].astype(str),
        errors='coerce'
    )
    anomalies_df = anomalies_df.sort_values('datetime_temp').drop(columns='datetime_temp')
else:
    anomalies_df = anomalies_df.sort_index()

# ------------------- ذخیره انومالی‌ها -------------------
anomalies_df.to_excel(output_file, index=False)

print(f"\nفایل انومالی‌ها با موفقیت ذخیره شد!")
print(f"   تعداد رکوردهای انومالی (شامل نویز و خوشه‌های کوچک): {len(anomalies_df):,} رکورد")
print(f"   مسیر فایل: {output_file}")
print("\nکار با موفقیت به پایان رسید!")

در حال خواندن فایل اکسل...

ستون‌های سنسور شناسایی شده (7 تا): ['AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344', 'AssetID_8346', 'AssetID_9286', 'AssetID_9287']

تعداد کل رکوردهای آماده: 10,319 رکورد

اجرای DBSCAN با eps=0.5 و min_samples=10...

                  خلاصه نتایج نهایی (با قوانین جدید)
کل داده‌ها                    : 10,319
نویز DBSCAN (برچسب -1)        : 482 رکورد
خوشه‌های کوچک (< {MIN_CLUSTER_SIZE} عضو): 4 خوشه → 47 رکورد
مجموع انومالی‌ها              : 529 رکورد ( 5.13%)
خوشه‌های اصلی (عادی ≥ {MIN_CLUSTER_SIZE} عضو): 9 خوشه
----------------------------------------------------------------------
خوشه اصلی    تعداد اعضا      درصد از کل
----------------------------------------------------------------------
خوشه 0             4,308 رکورد    ( 41.75%)
خوشه 6             2,789 رکورد    ( 27.03%)
خوشه 2             1,380 رکورد    ( 13.37%)
خوشه 12            1,211 رکورد    ( 11.74%)
خوشه 9                29 رکورد    (  0.28%)
خوشه 1                22 رکورد    (  0

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import os
from collections import Counter

# ------------------- تنظیمات -------------------
base_path = r"C:\BI"
input_file = os.path.join(base_path, "lube_oil_system_data_g11.xlsx")
output_anomalies = os.path.join(base_path, "lube_oil_system_anomalies_g11.xlsx")
output_full_model = os.path.join(base_path, "dbscan_model1.xlsx")

MIN_CLUSTER_SIZE = 15  # خوشه‌های کوچکتر از این → انومالی
eps = 0.5
min_samples = 10

# ------------------- خواندن داده -------------------
print("در حال خواندن داده‌ها...")
df = pd.read_excel(input_file)

# ستون‌های سنسور
sensor_columns = [col for col in df.columns if col.startswith('AssetID_') and str(col.split('_')[-1]).isdigit()]
print(f"ستون‌های سنسور: {len(sensor_columns)} تا")

id_columns = ['id', 'RecordDate', 'RecordTime']
optional_cols = ['unitID', 'TimeStamps', 'created_at', 'updated_at']
available_optional = [col for col in optional_cols if col in df.columns]

# ------------------- پیش‌پردازش -------------------
X = df[sensor_columns].copy()
X = X.fillna(X.mean())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ------------------- DBSCAN -------------------
print("اجرای DBSCAN...")
db = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
labels = db.fit_predict(X_scaled)

# ------------------- تحلیل خوشه‌ها -------------------
label_counts = Counter(labels)
large_cluster_labels = [lbl for lbl, cnt in label_counts.items() if lbl != -1 and cnt >= MIN_CLUSTER_SIZE]
small_cluster_labels = [lbl for lbl, cnt in label_counts.items() if lbl != -1 and cnt < MIN_CLUSTER_SIZE]
noise_mask = labels == -1

# نقاط متعلق به خوشه‌های بزرگ (برای محاسبه فاصله)
large_cluster_mask = np.isin(labels, large_cluster_labels)
X_large = X_scaled[large_cluster_mask]

# ------------------- محاسبه فاصله تا نزدیک‌ترین خوشه اصلی -------------------
print("محاسبه شدت انومالی (model_value)...")
if len(X_large) > 0:
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X_large)
    distances, _ = nbrs.kneighbors(X_scaled)
    distances = distances.flatten()
else:
    distances = np.zeros(len(X_scaled))  # اگر هیچ خوشه بزرگی نبود

# ------------------- ساخت model_value (0 تا 100) -------------------
anomaly_score = np.zeros(len(df))

for i, label in enumerate(labels):
    if label in large_cluster_labels:
        score = 0  # کاملاً نرمال
    else:
        # پایه: فاصله زیاد = امتیاز بالا
        dist_score = min(distances[i] / (eps * 3), 1.0) * 70  # حداکثر 70 از فاصله
        
        if label == -1:
            cluster_penalty = 30  # نویز خیلی غیرعادی
        elif label in small_cluster_labels:
            size = label_counts[label]
            cluster_penalty = max(10, 30 - size)  # خوشه 1 نفره → 29، خوشه 14 نفره → 16
        else:
            cluster_penalty = 0
        
        score = dist_score + cluster_penalty
        score = min(score, 100)  # حداکثر 100
    
    anomaly_score[i] = round(score, 2)

# ------------------- ساخت دیتافریم نهایی -------------------
result_df = df.copy()

# ستون DateTime
if 'RecordDate' in result_df.columns and 'RecordTime' in result_df.columns:
    result_df['DateTime'] = pd.to_datetime(
        result_df['RecordDate'].astype(str) + ' ' + result_df['RecordTime'].astype(str),
        errors='coerce'
    )
else:
    result_df['DateTime'] = pd.NaT

# اضافه کردن ستون‌های مدل
result_df['model_name'] = 'DBSCAN_Anomaly_Detection'
result_df['model_result'] = np.where(anomaly_score == 0, 'Normal', 'Anomaly')
result_df['model_value'] = anomaly_score

# مرتب‌سازی ستون‌ها به ترتیب خواسته شده
desired_columns = [
    'id', 'AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344',
    'AssetID_8346', 'AssetID_9286', 'AssetID_9287',
    'unitID', 'model_name', 'model_result', 'model_value',
    'DateTime', 'RecordDate', 'RecordTime', 'TimeStamps', 'created_at', 'updated_at'
]

# فقط ستون‌های موجود را نگه دار
final_columns = [col for col in desired_columns if col in result_df.columns]
# بقیه ستون‌های سنسور رو هم اضافه کن اگر نبودن
for col in sensor_columns:
    if col not in final_columns and col in result_df.columns:
        final_columns.insert(final_columns.index('unitID') if 'unitID' in final_columns else 8, col)

result_df = result_df[final_columns]

# مرتب‌سازی بر اساس زمان
result_df = result_df.sort_values('DateTime').reset_index(drop=True)

# ------------------- ذخیره فایل کامل مدل -------------------
print("در حال ذخیره فایل کامل مدل...")
result_df.to_excel(output_full_model, index=False)

# ------------------- ذخیره فقط انومالی‌ها (مثل قبل) -------------------
anomaly_mask_final = anomaly_score > 0
anomalies_df = result_df[anomaly_mask_final].copy()
anomalies_df.to_excel(output_anomalies, index=False)

# ------------------- خلاصه نهایی -------------------
n_anomalies = len(anomalies_df)
print("\n" + "="*70)
print("                  نتایج نهایی مدل DBSCAN")
print("="*70)
print(f"کل رکوردها            : {len(result_df):,}")
print(f"داده‌های عادی (Normal) : {len(result_df) - n_anomalies:,}")
print(f"داده‌های غیرعادی (Anomaly): {n_anomalies:,} ({n_anomalies/len(result_df)*100:.2f}%)")
print(f"بیشترین امتیاز انومالی: {anomaly_score.max():.1f}")
print(f"میانگین امتیاز انومالی در داده‌های غیرعادی: {anomaly_score[anomaly_score > 0].mean():.1f}")
print("="*70)
print(f"فایل کامل مدل ذخیره شد → {output_full_model}")
print(f"فایل فقط انومالی‌ها → {output_anomalies}")



در حال خواندن داده‌ها...
ستون‌های سنسور: 7 تا
اجرای DBSCAN...
محاسبه شدت انومالی (model_value)...
در حال ذخیره فایل کامل مدل...

                  نتایج نهایی مدل DBSCAN
کل رکوردها            : 10,927
داده‌های عادی (Normal) : 10,414
داده‌های غیرعادی (Anomaly): 513 (4.69%)
بیشترین امتیاز انومالی: 100.0
میانگین امتیاز انومالی در داده‌های غیرعادی: 70.5
فایل کامل مدل ذخیره شد → C:\BI\dbscan_model1.xlsx
فایل فقط انومالی‌ها → C:\BI\lube_oil_system_anomalies_g11.xlsx

موفقیت‌آمیز! حالا می‌تونی این فایل رو مستقیم در Power BI لود کنی و داشبورد بسازی
